# Face Detection and Recognition Training Pipeline

This pipeline outlines the overall steps for training and using a face recognition model:

## 0.1. Table of Contents
1. [Data Preparation](#1-import-libraries)
2. [Image Preprocessing (Face Detection and Cropping)](#2-face-detection)
3. [InceptionResnetV1 Model Initialization](#3-inceptionresnetv1)
4. [Model Training (Fine-tuning)](#4-fine-tuning)
5. [Testing and Inference](#5-inference-and-testing)
6. [TODO List](#6-todo-list)

## 0.2. Pipeline Overview

1. **[Data Preparation](#1-import-libraries)**: Face image data is organized in a specific directory structure, with each subdirectory representing an identity (person).

2. **[Image Preprocessing (Face Detection and Cropping)](#2-face-detection)**:
   * The MTCNN (Multi-task Cascaded Convolutional Networks) model is used to detect faces in each image.
   * Detected faces are cropped and aligned to a standard size (160x160 pixels), then saved to a new directory. This step ensures that only faces are fed into training, removing noise from the surrounding environment.

3. **[InceptionResnetV1 Model Initialization](#3-inceptionresnetv1)**:
   * An InceptionResnetV1 model is initialized. This model is initially trained on a large dataset (such as VGGFace2) and then fine-tuned for the specific number of classes (people) in your dataset.

4. **[Model Training (Fine-tuning)](#4-fine-tuning)**:
   * The cropped and normalized face data is split into training and validation sets.
   * The model is trained using Cross Entropy loss function and Adam optimizer.
   * The training process runs through multiple epochs, with performance monitoring on both training and validation sets.
   * After training, the model weights are saved.

5. **[Testing and Inference](#5-inference-and-testing)**:
   * The trained model is loaded back.
   * When a new image arrives, MTCNN is again used to detect and crop the face.
   * The cropped face is fed into the InceptionResnetV1 model to predict the identity and confidence of that prediction.

## 0.3. TODO List

### 📋 Implementation Tasks (4 TODOs to Complete)

#### 0.3.1. TODO 1: Dataset Information Function
**Location:** Section 1.2  
**Function:** `dataset_info(data_dir)`  
**Status:** ❌ Not Implemented  
**Priority:** High  

#### 0.3.2. TODO 2: Image Rotation Function
**Location:** Section 1.3  
**Function:** `rotate_image_180(image)`  
**Status:** ❌ Not Implemented  
**Priority:** Medium  

#### 0.3.3. TODO 3: Custom Accuracy Function
**Location:** Section 3.3  
**Function:** `custom_accuracy(outputs, targets)`  
**Status:** ❌ Not Implemented  
**Priority:** High  

#### 0.3.4. TODO 4: Custom Softmax Function
**Location:** Section 5.1  
**Function:** `custom_softmax(logits)`  
**Status:** ❌ Not Implemented  
**Priority:** High  

### 🧪 Test Functions with Assertions

Each TODO has comprehensive test functions with assertions:

#### Test Functions Available:
1. **`test_rotation_function()`** - Tests TODO 2 with assertions
2. **`test_custom_accuracy()`** - Tests TODO 3 with assertions  
3. **`test_custom_softmax()`** - Tests TODO 4 with assertions
4. **Edge case tests** for each function

#### Test Features:
- ✅ **Assertion-based validation** (no print statements)
- ✅ **Error handling and debugging**
- ✅ **Comparison with PyTorch implementations**
- ✅ **Edge case coverage**
- ✅ **Mathematical correctness verification**

### 📝 Implementation Order Recommendation:

1. **Start with TODO 1** (Dataset Info) - Essential for understanding your data
2. **Implement TODO 3** (Custom Accuracy) - Core ML metric needed for training
3. **Complete TODO 4** (Custom Softmax) - Required for inference
4. **Finish with TODO 2** (Image Rotation) - Data augmentation enhancement

### 🎯 Success Criteria:

- All test functions pass with assertions
- Functions match PyTorch implementations (where applicable)
- Code handles edge cases gracefully
- Mathematical properties are preserved
- Educational value is maintained (step-by-step implementations)

### 💡 Getting Started:

1. Run the test functions to see current status
2. Implement functions one by one
3. Use the comprehensive assert-based tests to verify correctness
4. Refer to the mathematical formulas and examples provided
5. Test edge cases thoroughly before moving to the next TODO

**Note:** All test functions use assertions for rigorous validation. This ensures your implementations are mathematically correct and robust.

## 1. Import Libraries

In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import os
import random

d:\anaconda\envs\uit-bootcamp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.1. Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [2]:
data_dir = 'Dataset/raw'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


### 1.2. TODO 1: Function dataset_info

In [4]:
def dataset_info(data_dir):
    """Print dataset information including number of classes and images per class
    TODO 1
    Args:
        data_dir (str): Path to the dataset directory (e.g., 'Dataset/raw')
    
    Expected Output:
        Should print information about the dataset in this format:
        
        Dataset Information for: Dataset/raw
        ==================================================
        Number of classes: 2
        Classes: ['vanhau', 'vantoan']
        
        Images per class:
        ------------------------------
          vanhau: 15 images
            Examples: OIP (1).jpg, OIP (2).jpg, OIP (3).jpg
            ... and 12 more
        
          vantoan: 12 images
            Examples: OIP (1).jpg, image1.jpg, image2.jpg
            ... and 9 more
        
        Total images in dataset: 27
        ==================================================
    
    Requirements:
        1. Check if the data_dir exists, if not print error message
        2. Find all subdirectories (these represent classes/people)
        3. Count total number of classes
        4. For each class, count how many image files (.jpg, .jpeg, .png, .bmp, .gif)
        5. Show first 3 image names as examples for each class
        6. Calculate and display total images across all classes
        7. Handle edge cases (empty directories, no classes found)
    
    Hints:
        - Use os.path.exists() to check if directory exists
        - Use os.listdir() to get directory contents
        - Use os.path.isdir() to check if item is a directory
        - Use str.lower().endswith() to check file extensions
        - Use len() to count items
        - Use sorted() to display classes in alphabetical order
    """
    pass

# Test the function with your dataset
dataset_info('Dataset/raw')

# def dataset_info(data_dir):
#     """Print dataset information including number of classes and images per class"""
#     # Step 1: Check if directory exists
#     if not os.path.exists(data_dir):
#         print(f"Error: Dataset directory '{data_dir}' not found.")
#         return
    
#     # Step 2: Find all subdirectories (classes)
#     items = os.listdir(data_dir)
#     classes = []
#     for item in items:
#         item_path = os.path.join(data_dir, item)
#         if os.path.isdir(item_path):
#             classes.append(item)
    
#     # Step 3: Handle no classes found
#     if not classes:
#         print(f"No class directories found in '{data_dir}'")
#         return
    
#     # Sort classes alphabetically
#     classes = sorted(classes)
    
#     # Step 4: Print header information
#     print(f"Dataset Information for: {data_dir}")
#     print("=" * 50)
#     print(f"Number of classes: {len(classes)}")
#     print(f"Classes: {classes}")
#     print()
#     print("Images per class:")
#     print("-" * 30)
    
#     # Step 5: Count images in each class
#     total_images = 0
#     image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
    
#     for class_name in classes:
#         class_path = os.path.join(data_dir, class_name)
        
#         # Get all files in class directory
#         files = os.listdir(class_path)
        
#         # Filter only image files
#         images = []
#         for file in files:
#             if file.lower().endswith(image_extensions):
#                 images.append(file)
        
#         num_images = len(images)
#         total_images += num_images
        
#         # Print class information
#         print(f"  {class_name}: {num_images} images")
        
#         # Show examples if images exist
#         if num_images > 0:
#             examples = images[:3]  # First 3 images
#             print(f"    Examples: {', '.join(examples)}")
#             if num_images > 3:
#                 print(f"    ... and {num_images - 3} more")
#         print()
    
#     # Step 6: Print total
#     print(f"Total images in dataset: {total_images}")
#     print("=" * 50)
    
# dataset_info('Dataset/raw')

### 1.3. TODO 2: Augmentation - Rotate Image 180 Degrees

In [5]:
def rotate_image_180(image):
    """
    TODO 2
    Rotate an image 180 degrees using only NumPy operations (for educational purposes)
    
    Args:
        image (numpy.ndarray): Input image with shape (height, width, channels) or (height, width)
                              Image values should be in range [0, 255] for uint8 or [0, 1] for float
    
    Returns:
        numpy.ndarray: Rotated image with same shape and data type as input
    
    Mathematical Concept:
        Rotating 180 degrees is equivalent to:
        - Flipping vertically (top-bottom) AND horizontally (left-right)
        - OR reversing both height and width dimensions
        
        For pixel at position (row, col):
        new_position = (height-1-row, width-1-col)
    
    Example:
        Original:     Rotated 180°:
        [1, 2, 3]  →  [9, 8, 7]
        [4, 5, 6]  →  [6, 5, 4] 
        [7, 8, 9]  →  [3, 2, 1]
    
    Your task:
        1. Get image dimensions (height, width)
        2. Create a new array with same shape and data type
        3. For each pixel, calculate its new position after 180° rotation
        4. Copy pixel values to new positions
        5. Return the rotated image
    
    Hints:
        - Use image.shape to get dimensions
        - Use np.zeros_like() to create array with same shape and dtype
        - Use nested loops or NumPy indexing to copy pixels
        - Remember: new_row = height - 1 - old_row
        - Remember: new_col = width - 1 - old_col
        - Handle both grayscale (2D) and color (3D) images
    """
    pass

# def rotate_image_180(image):
#     import numpy as np
    
#     # Method 1: Using nested loops (educational approach)
#     # Get image dimensions
#     if len(image.shape) == 2:  # Grayscale image
#         height, width = image.shape
#         rotated = np.zeros_like(image)
        
#         # Rotate each pixel
#         for row in range(height):
#             for col in range(width):
#                 new_row = height - 1 - row
#                 new_col = width - 1 - col
#                 rotated[new_row, new_col] = image[row, col]
                
#     elif len(image.shape) == 3:  # Color image
#         height, width, channels = image.shape
#         rotated = np.zeros_like(image)
        
#         # Rotate each pixel for all channels
#         for row in range(height):
#             for col in range(width):
#                 new_row = height - 1 - row
#                 new_col = width - 1 - col
#                 rotated[new_row, new_col] = image[row, col]
#     else:
#         raise ValueError("Image must be 2D (grayscale) or 3D (color)")
    
#     return rotated

# def rotate_image_180_optimized(image):
#     """
#     Optimized version using NumPy slicing (bonus implementation)
#     This is more efficient but less educational
#     """
#     import numpy as np
    
#     # Method 2: Using NumPy slicing (more efficient)
#     # Flip both dimensions: [::-1] reverses the array along that axis
#     if len(image.shape) == 2:  # Grayscale
#         return image[::-1, ::-1]
#     elif len(image.shape) == 3:  # Color
#         return image[::-1, ::-1, :]
#     else:
#         raise ValueError("Image must be 2D (grayscale) or 3D (color)")

In [6]:
def test_rotation_function():
    """Test function to verify rotation implementation using assertions"""
    import numpy as np
    
    print("Testing image rotation function with assertions...")
    print("=" * 50)
    
    # Test Case 1: Simple 3x3 matrix
    test_image = np.array([
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ], dtype=np.uint8)
    
    expected_rotated = np.array([
        [9, 8, 7],
        [6, 5, 4],
        [3, 2, 1]
    ], dtype=np.uint8)
    
    print("Test Case 1 - 3x3 Grayscale Image:")
    print("Original:")
    print(test_image)
    
    try:
        # Test manual implementation
        rotated_manual = rotate_image_180(test_image)
        print("\nRotated (Manual Implementation):")
        print(rotated_manual)
        
        # Assert tests
        assert rotated_manual is not None, "Function returned None - not implemented yet!"
        assert isinstance(rotated_manual, np.ndarray), f"Expected numpy array, got {type(rotated_manual)}"
        assert rotated_manual.shape == test_image.shape, f"Shape mismatch: expected {test_image.shape}, got {rotated_manual.shape}"
        assert rotated_manual.dtype == test_image.dtype, f"Data type mismatch: expected {test_image.dtype}, got {rotated_manual.dtype}"
        assert np.array_equal(rotated_manual, expected_rotated), "Rotation result doesn't match expected output"
        
        print("✅ Test Case 1: PASSED - All assertions successful!")
        
    except AssertionError as e:
        print(f"❌ Test Case 1: FAILED - {str(e)}")
        if 'not implemented' in str(e):
            print("Please implement the rotate_image_180() function.")
            return
        else:
            print("Expected:")
            print(expected_rotated)
            print("Got:")
            print(rotated_manual if 'rotated_manual' in locals() else "No result")
            return
    except Exception as e:
        print(f"❌ Test Case 1: ERROR - {str(e)}")
        print("Make sure you've implemented the rotate_image_180() function correctly.")
        return
    
    print("\n" + "=" * 50)
    
    # Test Case 2: Color image (3D array)
    test_color = np.array([
        [[255, 0, 0], [0, 255, 0]],     # Red, Green
        [[0, 0, 255], [255, 255, 0]]    # Blue, Yellow
    ], dtype=np.uint8)
    
    expected_color = np.array([
        [[255, 255, 0], [0, 0, 255]],   # Yellow, Blue
        [[0, 255, 0], [255, 0, 0]]      # Green, Red
    ], dtype=np.uint8)
    
    print("Test Case 2 - 2x2 Color Image:")
    print("Original shape:", test_color.shape)
    print("Original layout:")
    print("  Position (0,0): Red(255,0,0)    Position (0,1): Green(0,255,0)")
    print("  Position (1,0): Blue(0,0,255)   Position (1,1): Yellow(255,255,0)")
    
    try:
        rotated_color = rotate_image_180(test_color)
        print(f"\nRotated shape: {rotated_color.shape}")
        print("Expected layout after 180° rotation:")
        print("  Position (0,0): Yellow(255,255,0)  Position (0,1): Blue(0,0,255)")
        print("  Position (1,0): Green(0,255,0)     Position (1,1): Red(255,0,0)")
        
        # Assert tests for color image
        assert rotated_color is not None, "Function returned None for color image"
        assert isinstance(rotated_color, np.ndarray), f"Expected numpy array, got {type(rotated_color)}"
        assert rotated_color.shape == test_color.shape, f"Shape mismatch: expected {test_color.shape}, got {rotated_color.shape}"
        assert rotated_color.dtype == test_color.dtype, f"Data type mismatch: expected {test_color.dtype}, got {rotated_color.dtype}"
        assert len(rotated_color.shape) == 3, f"Expected 3D array for color image, got {len(rotated_color.shape)}D"
        assert np.array_equal(rotated_color, expected_color), "Color rotation result doesn't match expected output"
        
        print("✅ Test Case 2: PASSED - Color image rotation correct!")
        
    except AssertionError as e:
        print(f"❌ Test Case 2: FAILED - {str(e)}")
        print("Expected result:")
        print(expected_color)
        print("Your result:")
        print(rotated_color if 'rotated_color' in locals() else "No result")
        return
    except Exception as e:
        print(f"❌ Test Case 2: ERROR - {str(e)}")
        print("Make sure your function handles 3D color images correctly.")
        return
    
    print("\n🎉 ALL TESTS PASSED! Your rotation implementation is correct!")

# Test the rotation function
test_rotation_function()

def test_rotation_edge_cases():
    """Additional edge case tests with assertions"""
    import numpy as np
    
    print("\nTesting edge cases...")
    print("=" * 30)
    
    try:
        # Test Case 3: Single pixel
        single_pixel = np.array([[42]], dtype=np.uint8)
        rotated_single = rotate_image_180(single_pixel)
        assert np.array_equal(rotated_single, single_pixel), "Single pixel should remain unchanged"
        print("✅ Single pixel test: PASSED")
        
        # Test Case 4: 2x2 matrix
        test_2x2 = np.array([
            [1, 2],
            [3, 4]
        ], dtype=np.uint8)
        expected_2x2 = np.array([
            [4, 3],
            [2, 1]
        ], dtype=np.uint8)
        rotated_2x2 = rotate_image_180(test_2x2)
        assert np.array_equal(rotated_2x2, expected_2x2), "2x2 rotation failed"
        print("✅ 2x2 matrix test: PASSED")
        
        # Test Case 5: Different data types
        test_float = np.array([
            [0.1, 0.2],
            [0.3, 0.4]
        ], dtype=np.float32)
        expected_float = np.array([
            [0.4, 0.3],
            [0.2, 0.1]
        ], dtype=np.float32)
        rotated_float = rotate_image_180(test_float)
        assert np.allclose(rotated_float, expected_float), "Float array rotation failed"
        assert rotated_float.dtype == test_float.dtype, "Data type not preserved"
        print("✅ Float array test: PASSED")
        
        print("\n🎉 ALL EDGE CASE TESTS PASSED!")
        
    except AssertionError as e:
        print(f"❌ Edge case test FAILED: {str(e)}")
    except Exception as e:
        print(f"❌ Edge case test ERROR: {str(e)}")

# Run edge case tests
test_rotation_edge_cases()

def demonstrate_rotation_with_assertions():
    """Demonstrate rotation with assertion-based verification"""
    import numpy as np
    
    print("\nDemonstration with assertion-based verification:")
    print("=" * 50)
    
    try:
        # Create test pattern
        height, width = 100, 100
        test_pattern = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Create diagonal stripes
        for i in range(height):
            for j in range(width):
                if (i + j) % 20 < 10:
                    test_pattern[i, j] = [255, 0, 0]  # Red
                else:
                    test_pattern[i, j] = [0, 0, 255]  # Blue
        
        # Add white square in top-left corner
        test_pattern[10:30, 10:30] = [255, 255, 255]
        
        # Rotate the pattern
        rotated_pattern = rotate_image_180(test_pattern)
        
        # Assertions for verification
        assert rotated_pattern.shape == test_pattern.shape, "Shape should be preserved"
        assert rotated_pattern.dtype == test_pattern.dtype, "Data type should be preserved"
        
        # Verify white square moved from top-left to bottom-right
        original_white = np.all(test_pattern[10:30, 10:30] == [255, 255, 255], axis=2)
        rotated_white = np.all(rotated_pattern[70:90, 70:90] == [255, 255, 255], axis=2)
        
        assert np.all(original_white), "Original white square not found in expected position"
        assert np.all(rotated_white), "Rotated white square not found in expected position"
        
        # Verify corners are swapped correctly
        assert np.array_equal(test_pattern[0, 0], rotated_pattern[99, 99]), "Top-left to bottom-right swap failed"
        assert np.array_equal(test_pattern[99, 99], rotated_pattern[0, 0]), "Bottom-right to top-left swap failed"
        assert np.array_equal(test_pattern[0, 99], rotated_pattern[99, 0]), "Top-right to bottom-left swap failed"
        assert np.array_equal(test_pattern[99, 0], rotated_pattern[0, 99]), "Bottom-left to top-right swap failed"
        
        print("✅ Pattern creation and rotation: SUCCESS")
        print("✅ White square position verification: SUCCESS")
        print("✅ Corner pixel verification: SUCCESS")
        print("🎉 All demonstration assertions PASSED!")
        
    except AssertionError as e:
        print(f"❌ Demonstration FAILED: {str(e)}")
    except Exception as e:
        print(f"❌ Demonstration ERROR: {str(e)}")

# Run demonstration with assertions
demonstrate_rotation_with_assertions()

Testing image rotation function with assertions...
Test Case 1 - 3x3 Grayscale Image:
Original:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Rotated (Manual Implementation):
None
❌ Test Case 1: FAILED - Function returned None - not implemented yet!
Please implement the rotate_image_180() function.

Testing edge cases...
❌ Edge case test FAILED: Single pixel should remain unchanged

Demonstration with assertion-based verification:
❌ Demonstration ERROR: 'NoneType' object has no attribute 'shape'


In [7]:
class RandomRotation180:
    """Custom transform to randomly rotate images 180 degrees"""
    def __init__(self, probability=0.5):
        self.probability = probability
        # Check once if custom function is available
        self.use_custom = self._check_custom_function()
    
    def _check_custom_function(self):
        """Check if rotate_image_180 is implemented and working"""
        try:
            test_array = np.array([[1, 2], [3, 4]], dtype=np.uint8)
            result = rotate_image_180(test_array)
            return result is not None and isinstance(result, np.ndarray)
        except:
            return False
    
    def __call__(self, img):
        if random.random() < self.probability:
            if self.use_custom:
                # Use custom implementation
                try:
                    if hasattr(img, 'mode'):  # PIL Image
                        img_array = np.array(img)
                        rotated = rotate_image_180(img_array)
                        return Image.fromarray(rotated)
                    else:  # numpy array
                        return rotate_image_180(img)
                except:
                    # Fallback if custom function fails
                    pass
            
            # Use built-in rotation (fallback)
            if hasattr(img, 'mode'):  # PIL Image
                return img.rotate(180)
            else:  # numpy array
                if len(img.shape) == 2:  # Grayscale
                    return img[::-1, ::-1]
                elif len(img.shape) == 3:  # Color
                    return img[::-1, ::-1, :]
        return img

## 2. Face Detection

In [8]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [9]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

Batch 1 of 1

## 3. InceptionResNetV1

### 3.1. Init model

In [10]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

### 3.2. Setup Optimizer, Scheduler, ...

In [11]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

# Update transform to include both resize and rotation augmentation
trans = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize như bạn đã có
    RandomRotation180(probability=0.3),  # 30% chance to rotate 180 degrees
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

### 3.3. TODO 3: The Accuracy metric
#### Understanding Accuracy Calculation - Mathematical Formula

**Accuracy** is a fundamental metric in machine learning that measures the proportion of correct predictions.

#### Mathematical Formula:

```
accuracy = (number of correct predictions) / (total number of predictions)

accuracy = (1/N) × Σ(i=1 to N) [ŷᵢ == yᵢ]

where:
- N = total number of samples
- ŷᵢ = predicted class for sample i
- yᵢ = true class for sample i
- [condition] = indicator function (1 if true, 0 if false)
```

#### Step-by-Step Mathematical Process:

1. **Get Predicted Classes:**
   ```
   ŷᵢ = argmax(outputᵢ)
   where outputᵢ = [p₁, p₂, ..., pₖ] (model probabilities for k classes)
   ```

2. **Compare Predictions:**
   ```
   correctᵢ = {1 if ŷᵢ == yᵢ
              {0 if ŷᵢ ≠ yᵢ
   ```

3. **Calculate Final Accuracy:**
   ```
   accuracy = (Σ correctᵢ) / N = (correct_count) / N
   ```

#### Example:
```
Given: outputs = [[0.1,0.9], [0.8,0.2], [0.3,0.7], [0.6,0.4]]
       targets = [1, 0, 1, 0]

Step 1: ŷ = [argmax([0.1,0.9]), argmax([0.8,0.2]), argmax([0.3,0.7]), argmax([0.6,0.4])]
        ŷ = [1, 0, 1, 0]

Step 2: correct = [1==1, 0==0, 1==1, 0==0] = [1, 1, 1, 1]

Step 3: accuracy = (1+1+1+1)/4 = 4/4 = 1.0 = 100%
```

**Range:** accuracy ∈ [0, 1] where 0 = 0% correct, 1 = 100% correct

In [12]:
def custom_accuracy(outputs, targets):
    """
    TODO 3
    Calculate accuracy using simple math operations (for educational purposes)
    
    Args:
        outputs: Model predictions (tensor with shape [batch_size, num_classes])
        targets: True labels (tensor with shape [batch_size])
    
    Returns:
        accuracy: Float value between 0 and 1
    
    Example:
        If we have 4 samples with predictions and true labels:
        outputs = [[0.1, 0.9], [0.8, 0.2], [0.3, 0.7], [0.6, 0.4]]  # 2 classes
        targets = [1, 0, 1, 0]  # true labels
        
        Step 1: Find predicted class (highest value index)
        predicted = [1, 0, 1, 0]  # [0.9>0.1, 0.8>0.2, 0.7>0.3, 0.6>0.4]
        
        Step 2: Compare with true labels
        correct = [True, True, True, True]  # all predictions match targets
        
        Step 3: Calculate accuracy
        accuracy = 4/4 = 1.0 (100% correct)
    
    Your task:
        1. Convert PyTorch tensors to Python lists
        2. Find the predicted class for each sample (index of maximum value)
        3. Compare predictions with true labels
        4. Count correct predictions
        5. Calculate accuracy = correct_count / total_count
    
    Hints:
        - Use .tolist() to convert tensor to Python list
        - Use max() and list.index() to find the index of maximum value
        - Use sum() to count True values in a boolean list
        - Use len() to get total number of samples
    """
    pass

# def custom_accuracy(outputs, targets):
#     """Calculate accuracy using simple math operations (for educational purposes)"""
#     # Step 1: Convert PyTorch tensors to Python lists
#     outputs_list = outputs.tolist()
#     targets_list = targets.tolist()
    
#     # Step 2: Find predicted class for each sample (index of maximum value)
#     predicted_classes = []
#     for output_row in outputs_list:
#         # Find the index of maximum value
#         max_value = max(output_row)
#         predicted_class = output_row.index(max_value)
#         predicted_classes.append(predicted_class)
    
#     # Step 3: Compare predictions with true labels
#     correct_predictions = []
#     for i in range(len(predicted_classes)):
#         is_correct = predicted_classes[i] == targets_list[i]
#         correct_predictions.append(is_correct)
    
#     # Step 4: Count correct predictions
#     correct_count = sum(correct_predictions)  # sum() counts True as 1, False as 0
    
#     # Step 5: Calculate accuracy
#     total_count = len(targets_list)
#     accuracy = correct_count / total_count
    
#     return torch.tensor(accuracy)

Now test the function above :D xD

In [13]:
def test_custom_accuracy():
    """Test function to verify custom_accuracy implementation using assertions"""
    print("Testing custom_accuracy function with assertions...")
    print("=" * 50)
    
    # Test case 1: Perfect accuracy
    outputs_list = [
        [0.1, 0.9],  # predicted class 1, target should be 1
        [0.8, 0.2],  # predicted class 0, target should be 0  
        [0.3, 0.7],  # predicted class 1, target should be 1
        [0.9, 0.1]   # predicted class 0, target should be 0
    ]
    targets_list = [1, 0, 1, 0]
    expected_accuracy = 1.0  # 100% correct
    
    # Convert to tensors
    outputs_tensor = torch.tensor(outputs_list, dtype=torch.float32)
    targets_tensor = torch.tensor(targets_list, dtype=torch.long)
    
    print("Test Case 1 - Perfect Accuracy:")
    print(f"Outputs: {outputs_list}")
    print(f"Targets: {targets_list}")
    print(f"Expected: {expected_accuracy}")
    
    try:
        # Test your function
        accuracy = custom_accuracy(outputs_tensor, targets_tensor)
        print(f"Your result: {accuracy}")
        
        # Assert tests
        assert accuracy is not None, "Function returned None - not implemented yet!"
        assert isinstance(accuracy, (int, float, torch.Tensor)), f"Expected number or tensor, got {type(accuracy)}"
        
        # Convert to float for comparison
        acc_value = float(accuracy) if isinstance(accuracy, torch.Tensor) else accuracy
        assert isinstance(acc_value, (int, float)), f"Cannot convert result to float: {type(acc_value)}"
        assert 0.0 <= acc_value <= 1.0, f"Accuracy should be between 0 and 1, got {acc_value}"
        assert abs(acc_value - expected_accuracy) < 0.001, f"Expected {expected_accuracy}, got {acc_value}"
        
        print("✅ Test Case 1: PASSED - All assertions successful!")
        
    except AssertionError as e:
        print(f"❌ Test Case 1: FAILED - {str(e)}")
        if 'not implemented' in str(e):
            print("Please implement the custom_accuracy() function.")
            return
        else:
            print(f"Expected: {expected_accuracy}")
            print(f"Got: {accuracy if 'accuracy' in locals() else 'No result'}")
            return
    except Exception as e:
        print(f"❌ Test Case 1: ERROR - {str(e)}")
        print("Make sure you've implemented the custom_accuracy() function correctly.")
        return
    
    print("\n" + "=" * 50)
    
    # Test case 2: 50% accuracy
    outputs_list_2 = [
        [0.8, 0.2],  # predicted class 0, target is 1 (wrong)
        [0.1, 0.9],  # predicted class 1, target is 1 (correct)
        [0.6, 0.4],  # predicted class 0, target is 1 (wrong) 
        [0.3, 0.7]   # predicted class 1, target is 1 (correct)
    ]
    targets_list_2 = [1, 1, 1, 1]
    expected_accuracy_2 = 0.5  # 50% correct (2 out of 4)
    
    outputs_tensor_2 = torch.tensor(outputs_list_2, dtype=torch.float32)
    targets_tensor_2 = torch.tensor(targets_list_2, dtype=torch.long)
    
    print("Test Case 2 - 50% Accuracy:")
    print(f"Outputs: {outputs_list_2}")
    print(f"Targets: {targets_list_2}")
    print(f"Expected: {expected_accuracy_2}")
    
    try:
        accuracy_2 = custom_accuracy(outputs_tensor_2, targets_tensor_2)
        print(f"Your result: {accuracy_2}")
        
        # Assert tests for second case
        assert accuracy_2 is not None, "Function returned None for second test case"
        assert isinstance(accuracy_2, (int, float, torch.Tensor)), f"Expected number or tensor, got {type(accuracy_2)}"
        
        # Convert to float for comparison
        acc_value_2 = float(accuracy_2) if isinstance(accuracy_2, torch.Tensor) else accuracy_2
        assert isinstance(acc_value_2, (int, float)), f"Cannot convert result to float: {type(acc_value_2)}"
        assert 0.0 <= acc_value_2 <= 1.0, f"Accuracy should be between 0 and 1, got {acc_value_2}"
        assert abs(acc_value_2 - expected_accuracy_2) < 0.001, f"Expected {expected_accuracy_2}, got {acc_value_2}"
        
        print("✅ Test Case 2: PASSED - 50% accuracy calculation correct!")
        
    except AssertionError as e:
        print(f"❌ Test Case 2: FAILED - {str(e)}")
        print(f"Expected: {expected_accuracy_2}")
        print(f"Got: {accuracy_2 if 'accuracy_2' in locals() else 'No result'}")
        return
    except Exception as e:
        print(f"❌ Test Case 2: ERROR - {str(e)}")
        print("Make sure your function handles different accuracy scenarios correctly.")
        return
    
    print("\n" + "=" * 50)
    
    # Test case 3: Compare with PyTorch's built-in accuracy
    print("Test Case 3 - Comparison with PyTorch:")
    
    try:
        pytorch_acc_1 = training.accuracy(outputs_tensor, targets_tensor)
        pytorch_acc_2 = training.accuracy(outputs_tensor_2, targets_tensor_2)
        
        print(f"Test 1 - Your: {accuracy}, PyTorch: {pytorch_acc_1}")
        print(f"Test 2 - Your: {accuracy_2}, PyTorch: {pytorch_acc_2}")
        
        # Convert to float for comparison
        acc_1_val = float(accuracy) if isinstance(accuracy, torch.Tensor) else accuracy
        acc_2_val = float(accuracy_2) if isinstance(accuracy_2, torch.Tensor) else accuracy_2
        pytorch_1_val = float(pytorch_acc_1)
        pytorch_2_val = float(pytorch_acc_2)
        
        # Assert that our implementation matches PyTorch's
        assert abs(acc_1_val - pytorch_1_val) < 0.001, f"Test 1: Your {acc_1_val} != PyTorch {pytorch_1_val}"
        assert abs(acc_2_val - pytorch_2_val) < 0.001, f"Test 2: Your {acc_2_val} != PyTorch {pytorch_2_val}"
        
        print("✅ Test Case 3: PASSED - Matches PyTorch accuracy!")
        
    except AssertionError as e:
        print(f"❌ Test Case 3: FAILED - {str(e)}")
        print("Your implementation doesn't match PyTorch's accuracy.")
        print("Please check your logic and try again.")
        return
    except Exception as e:
        print(f"❌ Test Case 3: ERROR - {str(e)}")
        print("Error comparing with PyTorch accuracy.")
        return
    
    print("\n🎉 ALL TESTS PASSED! Your accuracy implementation is correct!")

# Test the custom_accuracy function with assertions
test_custom_accuracy()

def test_accuracy_edge_cases():
    """Additional edge case tests with assertions"""
    print("\nTesting accuracy edge cases...")
    print("=" * 30)
    
    try:
        # Test Case 4: Zero accuracy (all wrong)
        outputs_wrong = torch.tensor([
            [0.9, 0.1],  # predicted 0, target 1 (wrong)
            [0.8, 0.2],  # predicted 0, target 1 (wrong)
        ], dtype=torch.float32)
        targets_wrong = torch.tensor([1, 1], dtype=torch.long)
        
        accuracy_zero = custom_accuracy(outputs_wrong, targets_wrong)
        acc_zero_val = float(accuracy_zero) if isinstance(accuracy_zero, torch.Tensor) else accuracy_zero
        assert abs(acc_zero_val - 0.0) < 0.001, f"Expected 0.0 accuracy, got {acc_zero_val}"
        print("✅ Zero accuracy test: PASSED")
        
        # Test Case 5: Single sample
        outputs_single = torch.tensor([[0.2, 0.8]], dtype=torch.float32)
        targets_single = torch.tensor([1], dtype=torch.long)
        
        accuracy_single = custom_accuracy(outputs_single, targets_single)
        acc_single_val = float(accuracy_single) if isinstance(accuracy_single, torch.Tensor) else accuracy_single
        assert abs(acc_single_val - 1.0) < 0.001, f"Expected 1.0 accuracy, got {acc_single_val}"
        print("✅ Single sample test: PASSED")
        
        # Test Case 6: Multi-class (3 classes)
        outputs_multi = torch.tensor([
            [0.1, 0.2, 0.7],  # predicted class 2, target 2 (correct)
            [0.8, 0.1, 0.1],  # predicted class 0, target 0 (correct)
            [0.3, 0.6, 0.1],  # predicted class 1, target 1 (correct)
        ], dtype=torch.float32)
        targets_multi = torch.tensor([2, 0, 1], dtype=torch.long)
        
        accuracy_multi = custom_accuracy(outputs_multi, targets_multi)
        acc_multi_val = float(accuracy_multi) if isinstance(accuracy_multi, torch.Tensor) else accuracy_multi
        assert abs(acc_multi_val - 1.0) < 0.001, f"Expected 1.0 accuracy for multi-class, got {acc_multi_val}"
        print("✅ Multi-class test: PASSED")
        
        print("\n🎉 ALL EDGE CASE TESTS PASSED!")
        
    except AssertionError as e:
        print(f"❌ Edge case test FAILED: {str(e)}")
    except Exception as e:
        print(f"❌ Edge case test ERROR: {str(e)}")

# Run edge case tests
test_accuracy_edge_cases()

Testing custom_accuracy function with assertions...
Test Case 1 - Perfect Accuracy:
Outputs: [[0.1, 0.9], [0.8, 0.2], [0.3, 0.7], [0.9, 0.1]]
Targets: [1, 0, 1, 0]
Expected: 1.0
Your result: None
❌ Test Case 1: FAILED - Function returned None - not implemented yet!
Please implement the custom_accuracy() function.

Testing accuracy edge cases...
❌ Edge case test ERROR: unsupported operand type(s) for -: 'NoneType' and 'float'


The accuracy metrics above were merely our educational pseudo-implementation—a delightful learning exercise! Now, let us embrace the **official, battle-tested accuracy** that the pros use

In [14]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

## 4. Fine-tuning

In [15]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()

# Save the trained model after training completes
model_save_path = 'facenet_vantoan_vanhau.pth'
torch.save(resnet.state_dict(), model_save_path)
print(f'\nModel saved to: {model_save_path}')

# Save class names for inference
class_names_save_path = 'class_names.txt'
with open(class_names_save_path, 'w') as f:
    for class_name in dataset.classes:
        f.write(f"{class_name}\n")
print(f'Class names saved to: {class_names_save_path}')
print(f'Classes: {dataset.classes}')



Initial
----------
Valid |     1/1    | loss:    0.6188 | fps:    2.8988 | acc:    0.7500   

Epoch 1/8
----------
Train |     1/1    | loss:    0.6700 | fps:    0.9194 | acc:    0.5333   
Valid |     1/1    | loss:    0.7857 | fps:    2.7260 | acc:    0.5000   

Epoch 2/8
----------
Train |     1/1    | loss:    0.5873 | fps:    1.0631 | acc:    0.6667   
Valid |     1/1    | loss:    2.0004 | fps:    2.9687 | acc:    0.7500   

Epoch 3/8
----------
Train |     1/1    | loss:    1.1965 | fps:    1.1543 | acc:    0.6667   
Valid |     1/1    | loss:    0.9899 | fps:    3.5241 | acc:    0.7500   

Epoch 4/8
----------
Train |     1/1    | loss:    0.8660 | fps:    1.1095 | acc:    0.6667   
Valid |     1/1    | loss:    5.1225 | fps:    3.3416 | acc:    0.2500   

Epoch 5/8
----------
Train |     1/1    | loss:    0.3532 | fps:    1.2233 | acc:    0.8000   
Valid |     1/1    | loss:   28.4749 | fps:    3.2688 | acc:    0.2500   

Epoch 6/8
----------
Train |     1/1    | loss:    0.8

## 5. Inference and Testing
Test the trained model on sample images.

In [16]:
from PIL import Image
import matplotlib.pyplot as plt

# Load trained model for inference
def load_trained_model(model_path, num_classes, device):
    model = InceptionResnetV1(
        classify=True,
        pretrained='vggface2',
        num_classes=num_classes
    ).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model

# Initialize MTCNN for inference
mtcnn_inference = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

# Load class names with error handling
class_names_path = 'class_names.txt'
model_path = 'facenet_vantoan_vanhau.pth'

try:
    # Try to load class names from file
    with open(class_names_path, 'r') as f:
        class_names = [line.strip() for line in f.readlines()]
    print(f"Loaded class names from file: {class_names}")
except FileNotFoundError:
    # Fallback: use current dataset classes
    try:
        class_names = dataset.classes
        print(f"Using current dataset classes: {class_names}")
    except NameError:
        print("Error: No dataset or class names file found. Please run training first.")
        class_names = []

# Load trained model with error handling
if class_names and os.path.exists(model_path):
    try:
        model_inference = load_trained_model(model_path, len(class_names), device)
        print(f"Model loaded successfully for inference.")
    except Exception as e:
        print(f"Error loading model: {e}")
        model_inference = None
else:
    if not class_names:
        print("Cannot load model: No class names available.")
    else:
        print(f"Cannot load model: Model file not found at {model_path}")
        print("Please run the training cells first to create the model.")
    model_inference = None

Loaded class names from file: ['vanhau', 'vantoan']
Model loaded successfully for inference.


### 5.1. TODO 4: Understanding Softmax Function - Mathematical Formula

**Softmax** is an activation function that converts raw logits into a probability distribution.

#### Mathematical Formula:

```
For input vector z = [z₁, z₂, ..., zₖ], the softmax function is:

softmax(zᵢ) = e^(zᵢ) / Σ(j=1 to k) e^(zⱼ)

or in vector notation:
softmax(z) = [e^(z₁)/S, e^(z₂)/S, ..., e^(zₖ)/S]

where S = Σ(j=1 to k) e^(zⱼ) (normalization constant)
```

#### Mathematical Properties:

1. **Exponential Transformation:**
   ```
   expᵢ = e^(zᵢ) for each element i
   ```

2. **Normalization:**
   ```
   S = Σ(j=1 to k) e^(zⱼ) = e^(z₁) + e^(z₂) + ... + e^(zₖ)
   ```

3. **Final Probabilities:**
   ```
   pᵢ = e^(zᵢ) / S
   ```

#### Key Mathematical Properties:
```
1. Σ(i=1 to k) softmax(zᵢ) = 1  (probabilities sum to 1)
2. softmax(zᵢ) ∈ (0, 1)         (all values between 0 and 1)
3. softmax(zᵢ + c) = softmax(zᵢ) (translation invariant)
```

#### Example Calculation:
```
Input: z = [1.0, 2.0, 3.0]

Step 1: exp(z) = [e^1.0, e^2.0, e^3.0] = [2.718, 7.389, 20.086]

Step 2: S = 2.718 + 7.389 + 20.086 = 30.193

Step 3: softmax(z) = [2.718/30.193, 7.389/30.193, 20.086/30.193]
                   = [0.090, 0.245, 0.665]

Verification: 0.090 + 0.245 + 0.665 = 1.000 ✓
```

#### Why Softmax Works:
- **Exponential** emphasizes larger values (winner-take-most)
- **Normalization** ensures valid probability distribution
- **Differentiable** for gradient-based optimization

In [17]:
def custom_softmax(logits):
    """
    TODO 4
    Implement softmax function using basic math operations (for educational purposes)
    
    Args:
        logits (torch.Tensor): Input tensor with shape [batch_size, num_classes]
                              Contains raw model outputs (logits)
    
    Returns:
        torch.Tensor: Softmax probabilities with same shape as input
                     Each row sums to 1.0, values between 0 and 1
    
    Mathematical Formula:
        For each element logits[i][j]:
        softmax[i][j] = exp(logits[i][j]) / sum(exp(logits[i][k]) for all k)
    
    Example:
        Input:  logits = [[1.0, 2.0, 3.0]]  # 1 sample, 3 classes
        Step 1: exp_values = [[2.718, 7.389, 20.086]]
        Step 2: sum_exp = 2.718 + 7.389 + 20.086 = 30.193
        Step 3: softmax = [[0.090, 0.245, 0.665]]  # probabilities sum to 1.0
    
    Your task:
        1. Convert tensor to Python lists for easier manipulation
        2. For each sample (row), calculate exponential of each logit
        3. Sum all exponentials for that sample
        4. Divide each exponential by the sum to get probabilities
        5. Convert result back to tensor and return
    
    Hints:
        - Use .tolist() to convert tensor to Python lists
        - Use math.exp() to calculate exponential
        - Use sum() to calculate sum of exponentials
        - Use torch.tensor() to convert result back to tensor
        - Process each row (sample) separately
        - Make sure each row sums to approximately 1.0
    """
    pass

# def custom_softmax(logits):
#     """Implement softmax function using basic math operations"""
#     import math
    
#     # Step 1: Convert tensor to Python lists
#     logits_list = logits.tolist()
    
#     # Step 2: Calculate softmax for each sample
#     softmax_result = []
    
#     for sample_logits in logits_list:  # For each sample (row)
#         # Step 3: Calculate exponentials
#         exp_values = []
#         for logit in sample_logits:
#             exp_values.append(math.exp(logit))
        
#         # Step 4: Calculate sum of exponentials
#         sum_exp = sum(exp_values)
        
#         # Step 5: Calculate softmax probabilities
#         softmax_row = []
#         for exp_val in exp_values:
#             probability = exp_val / sum_exp
#             softmax_row.append(probability)
        
#         softmax_result.append(softmax_row)
    
#     # Step 6: Convert back to tensor
#     return torch.tensor(softmax_result, dtype=torch.float32)

Now test our implemented function :v

In [18]:
def test_custom_softmax():
    """Test function to verify custom_softmax implementation"""
    print("Testing custom_softmax function...")
    print("=" * 50)
    
    # Check if custom_softmax is implemented
    try:
        # Try a simple test first to see if function is implemented
        test_logits = torch.tensor([[1.0, 2.0]], dtype=torch.float32)
        test_result = custom_softmax(test_logits)
        
        # If we get here, function is implemented but might return None
        if test_result is None:
            print("❌ Function custom_softmax() returns None - not implemented yet!")
            print("Please implement the function according to the TODO instructions.")
            return
            
    except (NotImplementedError, TypeError, AttributeError):
        print("❌ Function custom_softmax() is not implemented yet!")
        print("Please implement the function according to the TODO instructions.")
        return
    except Exception as e:
        print(f"❌ Error in custom_softmax() implementation: {str(e)}")
        print("Please check your implementation and try again.")
        return
    
    print("✅ Function custom_softmax() is implemented! Running tests...")
    print()
    
    # Test case 1: Simple 2-class case
    logits_1 = torch.tensor([[1.0, 2.0]], dtype=torch.float32)
    
    try:
        # Test your function
        custom_result = custom_softmax(logits_1)
        pytorch_result = torch.nn.functional.softmax(logits_1, dim=1)
        
        print(f"Test Case 1 - Simple 2-class:")
        print(f"Input logits: {logits_1.tolist()}")
        print(f"Your result: {custom_result.tolist()}")
        print(f"PyTorch result: {pytorch_result.tolist()}")
        
        # Check if results are close
        if custom_result is None:
            print(f"Status: ❌ FAIL - Function returns None")
        elif isinstance(custom_result, torch.Tensor):
            # Check if values are close to PyTorch implementation
            if torch.allclose(custom_result, pytorch_result, atol=0.001):
                print(f"Status: ✅ PASS - Matches PyTorch softmax!")
            else:
                print(f"Status: ❌ FAIL - Results don't match PyTorch")
                
            # Check if probabilities sum to 1
            row_sum = custom_result.sum(dim=1).item()
            print(f"Row sum: {row_sum:.6f} (should be ~1.0)")
        else:
            print(f"Status: ❌ FAIL - Expected tensor, got {type(custom_result)}")
        print()
        
    except Exception as e:
        print(f"Test Case 1 - Error: {str(e)}")
        print("❌ FAIL - Exception occurred during test")
        print()
        return
    
    # Test case 2: Multi-class, multi-sample
    logits_2 = torch.tensor([
        [1.0, 2.0, 3.0],     # Sample 1
        [0.5, 1.5, 0.0]      # Sample 2
    ], dtype=torch.float32)
    
    try:
        custom_result_2 = custom_softmax(logits_2)
        pytorch_result_2 = torch.nn.functional.softmax(logits_2, dim=1)
        
        print(f"Test Case 2 - Multi-class, multi-sample:")
        print(f"Input logits: {logits_2.tolist()}")
        print(f"Your result: {custom_result_2.tolist()}")
        print(f"PyTorch result: {pytorch_result_2.tolist()}")
        
        # Check if results are close
        if custom_result_2 is None:
            print(f"Status: ❌ FAIL - Function returns None")
        elif isinstance(custom_result_2, torch.Tensor):
            if torch.allclose(custom_result_2, pytorch_result_2, atol=0.001):
                print(f"Status: ✅ PASS - Matches PyTorch softmax!")
            else:
                print(f"Status: ❌ FAIL - Results don't match PyTorch")
                
            # Check if each row sums to 1
            row_sums = custom_result_2.sum(dim=1)
            print(f"Row sums: {row_sums.tolist()} (each should be ~1.0)")
        else:
            print(f"Status: ❌ FAIL - Expected tensor, got {type(custom_result_2)}")
        print()
        
    except Exception as e:
        print(f"Test Case 2 - Error: {str(e)}")
        print("❌ FAIL - Exception occurred during test")
        print()
        return
    
    # Final comparison message
    try:
        if (custom_result is not None and custom_result_2 is not None and
            torch.allclose(custom_result, pytorch_result, atol=0.001) and
            torch.allclose(custom_result_2, pytorch_result_2, atol=0.001)):
            print("🎉 Congratulations! Your softmax implementation is correct!")
            print("You can now use it in the predict_image function.")
        else:
            print("❌ Implementation needs improvement. Please check the mathematical formula.")
            
    except Exception as e:
        print(f"Error in final comparison: {str(e)}")

# Test the custom_softmax function
test_custom_softmax()

Testing custom_softmax function...
❌ Function custom_softmax() returns None - not implemented yet!
Please implement the function according to the TODO instructions.


In [19]:
def predict_image(image_path, model, mtcnn, class_names, device):
    """Predict class of face in image"""
    try:
        # Step 1: Load and convert image to RGB
        img = Image.open(image_path).convert('RGB')
        
        # Step 2: Use MTCNN to detect and crop face
        img_cropped = mtcnn(img)
        
        # Step 3: Check if face was detected
        if img_cropped is None:
            return "No face detected", 0.0
        
        # Step 4: Preprocess for model input (add batch dimension and move to device)
        img_cropped = img_cropped.unsqueeze(0).to(device)
        
        # Step 5: Run inference with trained model
        with torch.no_grad():
            outputs = model(img_cropped)
            
            # Step 6: Apply softmax to get probabilities
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            
            # Step 7: Find class with highest probability
            confidence, predicted = torch.max(probabilities, 1)
        
        # Step 8: Get predicted class name and confidence score
        predicted_class = class_names[predicted.item()]
        confidence_score = confidence.item()
        
        return predicted_class, confidence_score
        
    except Exception as e:
        # Step 9: Handle exceptions gracefully
        return f"Error: {str(e)}", 0.0


def test_predict_image():
    """Test function to verify predict_image implementation"""
    print("Testing predict_image function...")
    print("=" * 50)
    
    # Check if predict_image is implemented
    try:
        # Create dummy inputs for testing
        test_image_path = "dummy_path.jpg"  # This will cause an error, which is expected for testing
        dummy_model = None
        dummy_mtcnn = None
        dummy_class_names = ["class1", "class2"]
        dummy_device = torch.device('cpu')
        
        test_result = predict_image(test_image_path, dummy_model, dummy_mtcnn, dummy_class_names, dummy_device)
        
        # If we get here, function is implemented but might return None
        if test_result is None:
            print("❌ Function predict_image() returns None - not implemented yet!")
            print("Please implement the function according to the TODO instructions.")
            return
        
        # Check if it returns a tuple with 2 elements
        if not isinstance(test_result, tuple) or len(test_result) != 2:
            print("❌ Function should return a tuple with 2 elements: (predicted_class, confidence)")
            print("Please check your implementation.")
            return
            
    except (NotImplementedError, TypeError, AttributeError):
        print("❌ Function predict_image() is not implemented yet!")
        print("Please implement the function according to the TODO instructions.")
        return
    except Exception as e:
        # This is expected since we're using dummy inputs
        if "dummy_path.jpg" in str(e) or "NoneType" in str(e):
            print("✅ Function predict_image() is implemented! (Error handling works correctly)")
            print("Function correctly handles invalid inputs and returns error messages.")
        else:
            print(f"❌ Unexpected error in predict_image() implementation: {str(e)}")
            print("Please check your implementation and try again.")
            return
    
    print("\n🎉 predict_image() function structure is correct!")
    print("💡 To fully test this function, you need:")
    print("   1. A trained model loaded")
    print("   2. MTCNN initialized") 
    print("   3. Valid image files")
    print("   4. Class names list")
    print("\nOnce training is complete, this function will be tested automatically.")

# Test the predict_image function
test_predict_image()

Testing predict_image function...

🎉 predict_image() function structure is correct!
💡 To fully test this function, you need:
   1. A trained model loaded
   2. MTCNN initialized
   3. Valid image files
   4. Class names list

Once training is complete, this function will be tested automatically.


In [20]:
def test_sample_images():
    """Test model on sample images from both classes"""
    # Check if predict_image function is implemented first
    try:
        # Test with dummy inputs to see if function is implemented
        dummy_result = predict_image("dummy.jpg", None, None, ["test"], torch.device('cpu'))
        
        # If we get here and result is None, function is not implemented
        if dummy_result is None:
            print("❌ Function predict_image() is not implemented yet!")
            print("Please implement the predict_image() function according to the TODO 3 instructions.")
            return []
            
    except (NotImplementedError, TypeError, AttributeError):
        print("❌ Function predict_image() is not implemented yet!")
        print("Please implement the predict_image() function according to the TODO 3 instructions.")
        return []
    except Exception as e:
        # This is expected for dummy inputs - function is implemented
        pass
    
    # Check if model and class names are available
    if model_inference is None:
        print("Error: Model not loaded. Cannot run inference.")
        return []
    
    if not class_names:
        print("Error: No class names available.")
        return []
    
    print("✅ predict_image() function is implemented! Running tests...")
    print()
    
    test_results = []
    
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        if os.path.exists(class_dir):
            images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            if not images:
                print(f"No images found in {class_dir}")
                continue
            
            # Test first few images from each class
            for img_file in images[:3]:
                img_path = os.path.join(class_dir, img_file)
                predicted_class, confidence = predict_image(
                    img_path, model_inference, mtcnn_inference, class_names, device
                )
                
                result = {
                    'true_class': class_name,
                    'predicted_class': predicted_class,
                    'confidence': confidence,
                    'correct': predicted_class == class_name,
                    'image_path': img_path
                }
                test_results.append(result)
                
                status = "✅" if result['correct'] else "❌"
                print(f"{status} {class_name} -> {predicted_class} (conf: {confidence:.3f})")
        else:
            print(f"Directory not found: {class_dir}")
    
    # Summary
    if test_results:
        correct_predictions = sum(1 for r in test_results if r['correct'])
        accuracy = correct_predictions / len(test_results)
        print(f"\nTest Results: {correct_predictions}/{len(test_results)} correct ({accuracy*100:.1f}%)")
    else:
        print("No test results available.")
    
    return test_results

# Run tests with comprehensive error handling
print("Testing model on sample images...")
print("=" * 50)

# Check all prerequisites
if model_inference is not None and class_names:
    test_results = test_sample_images()
else:
    print("Cannot run tests: Model or class names not available.")
    print("Please ensure training has completed successfully.")
    print()
    
    # Also check if predict_image is implemented
    try:
        dummy_result = predict_image("dummy.jpg", None, None, ["test"], torch.device('cpu'))
        if dummy_result is None:
            print("Additionally: predict_image() function is not implemented.")
    except (NotImplementedError, TypeError, AttributeError):
        print("Additionally: predict_image() function is not implemented.")
    except:
        print("predict_image() function appears to be implemented.")

Testing model on sample images...
✅ predict_image() function is implemented! Running tests...

❌ vanhau -> vantoan (conf: 1.000)
❌ vanhau -> vantoan (conf: 1.000)
❌ vanhau -> vantoan (conf: 1.000)
✅ vantoan -> vantoan (conf: 1.000)
✅ vantoan -> vantoan (conf: 1.000)
✅ vantoan -> vantoan (conf: 1.000)

Test Results: 3/6 correct (50.0%)
